In [6]:
from sklearn.externals import joblib 
import re
from nltk.stem import PorterStemmer,SnowballStemmer
from nltk.corpus import stopwords
  
# Load the model from the file 
knn= joblib.load('amazon-review-analysis-knn.pkl')
model = joblib.load('amazon-review-analysis-w2v.pkl') 
w2v_words=dict(model.wv.vocab)
  
# Use the loaded model to make predictions 
def clean_html(sentence):
    x=re.compile('<.*?>')
    cleantext=re.sub(x,' ',sentence)
    return cleantext

def clean_punc(sentence):
    cleaned=re.sub(r'[?|!|\'|"|#|.|,|)|(|\|/]',r' ',sentence)
    return cleaned


sno=SnowballStemmer('english')
stop=set(stopwords.words('english'))

def sent_to_vec(r):
    r=clean_html(r)
    r=clean_punc(r)
    test_review=[]
    for w in r.split():
        if(w.isalpha() and len(w)>2 and w.lower() not in stop):
            w=(sno.stem(w.lower()))
            test_review.append(w)

    import numpy as np
    vec=np.zeros(100)
    cnt_words=0

    for word in test_review:
        if word in w2v_words:
            vec+=model.wv[word]
            cnt_words+=1

    if cnt_words!=0:
        vec/=cnt_words
    
    return vec
    
r=input("Enter a Product Review")
vec=sent_to_vec(r)
        
probs=knn.predict_proba([vec])[0]
ans=""
if knn.predict([vec])==1:
    ans+="Positive Review with "+str(probs[1])+" probabilty"
else:
    ans+="Negative Review with "+str(probs[0])+" probability"

print(ans)
    

Enter a Product Reviewthis is a good product
Positive Review with 0.96 probabilty
